In [37]:
import pandas as pd
import numpy as np
import matplotlib as plt
import time



In [38]:
defect= pd.read_csv("defects.csv")

defect=defect.sort_values(by='x').reset_index(drop=True)

result_dict = dict(zip(defect['x'], defect['class']))

defect.head(16)

,x,class
0,0.700561,a
1,0.848185,a
2,1.769825,a
3,2.207230,b
4,2.313038,a
5,3.701164,a
6,3.998342,b
7,8.757472,c
8,9.122731,c
9,9.956196,c


In [39]:
def get_keys_less_than_value(input_dict, value, value2):
    return [input_dict[key] for key in input_dict.keys() if key > value and key < value2]


def get_classes_below_threshold(df, threshold):
    return df.loc[df['x'] < threshold, 'class'].tolist()


def get_values_defect(input_dict, value1, value2):
    accumulated_values = ''
    for key in input_dict.keys():
        if key > value1 and key <= value2:
            accumulated_values += input_dict[key]  # Concatenate the values meeting the condition
    
    return accumulated_values.strip()  # Remove trailing space, if any




In [40]:

start_time = time.perf_counter()

a = get_values_defect(result_dict, 100,110)



end_time = time.perf_counter()
execution_time1 = end_time - start_time
print(f"Execution time: {execution_time1} seconds",a)


Execution time: 0.0003990000000158034 seconds cabaca


In [41]:


start_time = time.perf_counter()

# Your code
a = get_keys_less_than_value(result_dict, 100,110)

end_time = time.perf_counter()
execution_time2 = end_time - start_time
print(f"Execution time: {execution_time2} seconds",a)

Execution time: 0.00041059999966819305 seconds ['c', 'a', 'b', 'a', 'c', 'a']


In [42]:
print(f"Execution time 1 minus 2: {execution_time1/execution_time2} seconds")

Execution time 1 minus 2: 0.9717486613205951 seconds


In [43]:
class Biscuit:
    def __init__(self, length, value, max_a, max_b, max_c):
        self.length = length
        self.value = value
        self.max_defects = {'a': max_a, 'b': max_b, 'c': max_c}

# Creating instances of the Biscuit class for each type of biscuit
biscuit_0 = Biscuit(length=4, value=6, max_a=4, max_b=2, max_c=3)
biscuit_1 = Biscuit(length=8, value=12, max_a=5, max_b=4, max_c=4)
biscuit_2 = Biscuit(length=2, value=1, max_a=1, max_b=2, max_c=1)
biscuit_3 = Biscuit(length=5, value=8, max_a=2, max_b=3, max_c=2)
biscuit_4 = Biscuit(length=1, value=0, max_a=float('inf'), max_b=float('inf'), max_c=float('inf'))
#biscuit_5 = Biscuit(length=100, value=8, max_a=0, max_b=0, max_c=0)

# Store all biscuits in a list
biscuits = [biscuit_0, biscuit_1, biscuit_2, biscuit_3, biscuit_4,biscuit_5]

# Accessing details of a specific biscuit

for key, value in biscuits[2].max_defects.items():
    print(f'Type of {key}: {type(value)}')

for key, value in biscuits[2].max_defects.items():
    print(key,value)

    


Type of a: <class 'int'>
Type of b: <class 'int'>
Type of c: <class 'int'>
a 1
b 2
c 1


In [44]:
class Dough_Roll:
    def __init__(self, max_length,defect, biscuit_order=None,filled_length=0):
        if biscuit_order is None:
            biscuit_order = []
        self.defect = defect
        self.max_length = max_length
        self.biscuit_order = biscuit_order
        self.filled_length = filled_length


    def can_i_add(self, biscuit,On_current_self=True, filled_length=None):
        output = True
        if On_current_self==True:
            #check if length of biscuit is consistent with max_length of the dough roll
            if biscuit.length+self.filled_length>self.max_length:
                output = False
            #check if the number of defect is acceptable for each defect
            for key, defect_acceptable in biscuit.max_defects.items():
                if defect_acceptable-get_values_defect(self.defect,self.filled_length,biscuit.length+self.filled_length).count(key)<0:
                    output = False
        else:
            #check if the number of defect is acceptable for each defect
            for key, defect_acceptable in biscuit.max_defects.items():
                if defect_acceptable-get_values_defect(self.defect,filled_length,biscuit.length+filled_length).count(key)<0:
                    output = False
        return output
    

    def fitness_evaluator(self,biscuit_list,ref_biscuit_order_index=0, filled_length=0):
        #check if the index is within bounds 
        if len(self.biscuit_order)>ref_biscuit_order_index:
            current_biscuit=biscuit_list[self.biscuit_order[ref_biscuit_order_index]]
            #check if the next biscuit is acceptable
            if self.can_i_add(current_biscuit, False, filled_length):
                return (current_biscuit.value 
                        + self.fitness_evaluator(biscuit_list,ref_biscuit_order_index+1, filled_length+current_biscuit.length))
        return 0
    
    def greedy_add(self,biscuit_list):
        output = False
        for biscuit in biscuit_list:
            if self.can_i_add(biscuit)==True:
                output = True
        if output == True:
            max_input = biscuit_list.index(max(biscuit_list,key=lambda i: (i.value / i.length)*self.can_i_add(i)+self.can_i_add(i)))
            self.biscuit_order.append(max_input)
            self.filled_length+=biscuit_list[max_input].length
        return output
    





In [45]:
greedy_dough = Dough_Roll(500,result_dict)

while greedy_dough.filled_length<greedy_dough.max_length:
    greedy_dough.greedy_add(biscuits)
print(greedy_dough.fitness_evaluator(biscuits),greedy_dough.filled_length)

1
1
3
0
0
3
2
0
0
0
2
0
3
3
0
3
0
3
1
0
4
2
4
0
3
0
0
4
4
4
0
0
3
3
0
0
3
3
2
4
4
3
4
2
4
3
3
0
2
4
4
2
1
3
3
3
3
3
3
3
3
3
4
4
0
0
3
3
3
3
3
0
0
3
3
1
1
3
0
3
3
0
2
3
3
3
3
1
0
0
0
0
3
3
3
3
0
3
3
3
3
3
0
0
0
3
0
3
0
3
3
3
3
0
4
4
0
3
3
2
4
735 500
